## Orders data - Complex JSON ( have bugs )


## Handling bronze schema

In [0]:
-- There are corrupted records in this json file so for bronze schema we store is at text view.
select * from JSON.`/Volumes/gizmobox/landing/operational_data/orders/`

In [0]:
select * from text.`/Volumes/gizmobox/landing/operational_data/orders/`

In [0]:
create or replace view gizmobox.bronze.v_bronze_orders as
select * from text.`/Volumes/gizmobox/landing/operational_data/orders/`;

## Handling Silver schema 
Some transformations and array explode 

In [0]:
select * from gizmobox.bronze.v_bronze_orders;

In [0]:
select value:order_id , value from gizmobox.bronze.v_bronze_orders;

In [0]:
create or replace temporary view v_corrected_orders
as
select regexp_replace(value,'"order_date": ([0-9]{4}-[0-9]{2}-[0-9]{2})','"order_date":"\$1"') as value from gizmobox.bronze.v_bronze_orders;

To convert this string value to json .
- first get the schema of json using schema_of_json() functions
- through function from_json() convert string to json by giving schema into param

In [0]:
select schema_of_json(value) from v_corrected_orders;

In [0]:
select from_json(value,'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') from v_corrected_orders;

In [0]:
create or replace table orders_json
as
select from_json(value,'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as value from v_corrected_orders;

In [0]:
select * from orders_json;

In [0]:
create or replace temporary view tv_orders
as
select 
  value.customer_id as customer_id,
  value.order_id as order_id,
  cast(value.order_date as date) as order_date,
  value.order_status as order_status,
  value.payment_method as payment_method,
  value.total_amount as total_amount,
  cast(value.transaction_timestamp as timestamp) as transaction_timestamp,
  array_distinct(value.items) as items --removing duplicate records in the array
from orders_json
order by order_id;

In [0]:
create or replace temporary view tv_exploaded_orders
as
select 
  customer_id,
  order_id,
  order_date,
  order_status,
  payment_method,
  total_amount,
  transaction_timestamp,
  explode(items) as items
from tv_orders;

In [0]:
select
  customer_id,
  order_id,
  order_date,
  order_status,
  payment_method,
  total_amount,
  transaction_timestamp,
  items.item_id as item_id,
  items.name as name,
  items.price as price,
  items.quantity as quantity,
  items.category as category,
  items.details.brand as brand,
  items.details.color as color
from tv_exploaded_orders;

In [0]:
create table if not exists gizmobox.silver.orders
as
select
  customer_id,
  order_id,
  order_date,
  order_status,
  payment_method,
  total_amount,
  transaction_timestamp,
  items.item_id as item_id,
  items.name as name,
  items.price as price,
  items.quantity as quantity,
  items.category as category,
  items.details.brand as brand,
  items.details.color as color
from tv_exploaded_orders;

In [0]:
select * from gizmobox.silver.orders;